In [2]:
import pandas as pd
import numpy as np
from dateutil import relativedelta
from datetime import datetime
import random

In [9]:

# Criando uma base de dados de exemplo
data = {
    'ID do Cliente': [random.randint(0, 10) for _ in range(100)],
    'Data': pd.date_range(start='2002-01-01', periods=100, freq='1M'),
    'Receita': [random.randint(10, 100) * 10 for _ in range(100)]
}


df = pd.DataFrame(data)

In [10]:
df.head()

,ID do Cliente,Data,Receita
0,6,2002-01-31,490
1,7,2002-02-28,930
2,3,2002-03-31,580
3,7,2002-04-30,470
4,10,2002-05-31,830


In [11]:
# Passo 2: Calcular a métrica de interesse (Receita Média por Cliente)
df['Data'] = pd.to_datetime(df['Data'])
df['Mês'] = df['Data'].dt.to_period('M')
df['Receita'] = df['Receita']

# Passo 3: Criar cohorts com base na data da primeira transação
df['Cohort'] = df.groupby('ID do Cliente')['Data'].transform('min').dt.to_period('M')

# Passo 4: Calcular a métrica para cada cohort
cohort_data = df.groupby(['Cohort', 'Mês'])['Receita'].mean().reset_index()

# Calcular o número de meses em relação ao cohort
cohort_data['Mês'] = (cohort_data['Mês'] - cohort_data['Cohort']).apply(lambda x: x.n)

# Passo 5: Criar a tabela de cohort
pivot_cohort_data = cohort_data.pivot(index='Cohort', columns='Mês', values='Receita')
pivot_cohort_data = pivot_cohort_data.round(2)  # Arredonde os valores para duas casas decimais
pivot_cohort_data.fillna('-', inplace=True)  # Substitua valores NaN por '-' para melhor legibilidade


print(pivot_cohort_data)
pivot_cohort_data.to_csv('cohort_teste.txt', sep='\t', index=True)
print(df)


Mês         0      2      4      5      7      8      11     13     14     16  \
Cohort                                                                          
2002-01  490.0      -      -      -      -      -  840.0  380.0      -      -   
2002-02  930.0  470.0  610.0      -  820.0      -      -      -      -      -   
2002-03  580.0      -      -      -      -      -      -      -      -      -   
2002-05  830.0      -      -      -      -      -      -      -  150.0  230.0   
2002-07  520.0      -      -      -      -      -  510.0      -      -      -   
2002-08  240.0  580.0      -      -      -      -      -      -      -      -   
2002-11  250.0      -      -      -      -      -      -      -      -      -   
2003-01  640.0      -  420.0      -      -      -      -  100.0      -      -   
2003-03  490.0      -      -  840.0      -  410.0      -      -      -      -   
2003-04  810.0      -      -      -      -  530.0      -      -      -      -   
2003-10  660.0      -      -